# STEP 2: Generate Road Networks (500×500m)
## Procedural Generation Based on Real City Metrics

**Goal**: Generate road networks that match the space syntax metrics extracted from real cities in STEP 1.

**Process**:
1. Load target metrics from STEP 1 analysis
2. Initialize road network (grid-based or organic growth)
3. Optimize network to match target metrics
4. Segment parcels from road network
5. Identify building block placement locations
6. Analyze generated network with space syntax
7. Create visualizations and comparisons

**Target Cities**: London, Berlin, Belgrade, Torino

**Output**:
- Generated road networks (GeoJSON)
- Parcel boundaries (GeoJSON)
- Building block locations (GeoJSON)
- Space syntax metrics comparison
- Visualizations (PNG + SVG)

## 1. Setup & Configuration

In [ ]:
# Imports
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap, Normalize
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, box
from shapely.ops import unary_union, polygonize, linemerge
from shapely.affinity import translate, rotate, scale
import json
from pathlib import Path
import warnings
from scipy.optimize import minimize
from scipy.spatial import Voronoi, voronoi_plot_2d
import random

warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("✓ Libraries imported successfully")

In [ ]:
# Configuration
CITIES = {
    'london': {'name': 'London, UK', 'color': '#E74C3C'},
    'berlin': {'name': 'Berlin, Germany', 'color': '#3498DB'},
    'belgrade': {'name': 'Belgrade, Serbia', 'color': '#2ECC71'},
    'torino': {'name': 'Torino, Italy', 'color': '#F39C12'}
}

# Generation parameters
AREA_SIZE = 500  # meters (500×500m)
RADIUS = AREA_SIZE / 2

# Road generation parameters
MIN_BLOCK_SIZE = 40  # meters
MAX_BLOCK_SIZE = 120  # meters
ROAD_WIDTH = 10  # meters (for visualization)
OPTIMIZATION_ITERATIONS = 100

# Parcel parameters
MIN_PARCEL_AREA = 500  # m²
MAX_PARCEL_AREA = 10000  # m²

# Output paths
INPUT_DIR = Path('outputs/metrics')
OUTPUT_DIR = Path('outputs_generated')
GEOJSON_DIR = OUTPUT_DIR / 'geojson'
VIZ_PNG_DIR = OUTPUT_DIR / 'visualizations' / 'png'
VIZ_SVG_DIR = OUTPUT_DIR / 'visualizations' / 'svg'
METRICS_DIR = OUTPUT_DIR / 'metrics'

for d in [GEOJSON_DIR, VIZ_PNG_DIR, VIZ_SVG_DIR, METRICS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("✓ Configuration complete")
print(f"  Generating {len(CITIES)} networks")
print(f"  Area size: {AREA_SIZE}×{AREA_SIZE}m")
print(f"  Output: {OUTPUT_DIR.absolute()}")

## 2. Load Target Metrics from STEP 1

In [ ]:
# Load metrics from STEP 1
print("Loading target metrics from STEP 1...")
print("="*60)

with open(INPUT_DIR / 'urban_metrics.json', 'r') as f:
    step1_data = json.load(f)

target_metrics = step1_data['urban_metrics']

# Display loaded metrics
print("\nTarget Metrics Loaded:")
for city_key, metrics in target_metrics.items():
    print(f"\n{metrics['name']}:")
    print(f"  Nodes: {metrics['nodes']['total_count']}")
    print(f"  Edges: {metrics['edges']['total_count']}")
    print(f"  Total Length: {metrics['edges']['total_length_m']:.0f}m")
    print(f"  Avg Segment: {metrics['edges']['segment_length_distribution']['mean']:.1f}m")
    print(f"  Buildings: {metrics['buildings']['total_count']}")
    print(f"  Parcels: {metrics['parcels']['total_count']}")

print("\n" + "="*60)
print("✓ Target metrics loaded successfully")

## 3. Road Network Initialization

Initialize road networks using different strategies for each city style.

In [ ]:
def create_grid_network(size, spacing, offset=(0, 0)):
    """
    Create a regular grid network.
    
    Args:
        size: Area size (meters)
        spacing: Grid spacing (meters)
        offset: (x, y) offset for grid origin
    
    Returns:
        NetworkX graph with positions
    """
    G = nx.Graph()
    
    # Calculate grid dimensions
    half_size = size / 2
    x_min, x_max = -half_size + offset[0], half_size + offset[0]
    y_min, y_max = -half_size + offset[1], half_size + offset[1]
    
    # Create grid points
    x_coords = np.arange(x_min, x_max + spacing, spacing)
    y_coords = np.arange(y_min, y_max + spacing, spacing)
    
    node_id = 0
    pos = {}
    
    # Create horizontal lines
    for y in y_coords:
        prev_node = None
        for x in x_coords:
            pos[node_id] = (x, y)
            if prev_node is not None:
                G.add_edge(prev_node, node_id, length=spacing)
            prev_node = node_id
            node_id += 1
    
    # Create vertical lines
    for i, x in enumerate(x_coords):
        for j in range(len(y_coords) - 1):
            n1 = i + j * len(x_coords)
            n2 = i + (j + 1) * len(x_coords)
            if n1 in G.nodes and n2 in G.nodes:
                G.add_edge(n1, n2, length=spacing)
    
    nx.set_node_attributes(G, pos, 'pos')
    return G


def create_organic_network(size, n_nodes, seed_points=5):
    """
    Create an organic network using growth simulation.
    
    Args:
        size: Area size (meters)
        n_nodes: Target number of nodes
        seed_points: Initial seed points for growth
    
    Returns:
        NetworkX graph with positions
    """
    G = nx.Graph()
    half_size = size / 2
    
    # Initialize with seed points
    pos = {}
    for i in range(seed_points):
        angle = 2 * np.pi * i / seed_points
        r = np.random.uniform(0, half_size * 0.3)
        x = r * np.cos(angle)
        y = r * np.sin(angle)
        pos[i] = (x, y)
        G.add_node(i)
    
    # Grow network organically
    node_id = seed_points
    while node_id < n_nodes:
        # Select random existing node
        parent = np.random.choice(list(G.nodes()))
        px, py = pos[parent]
        
        # Add new node nearby
        angle = np.random.uniform(0, 2 * np.pi)
        distance = np.random.uniform(30, 80)  # meters
        
        x = px + distance * np.cos(angle)
        y = py + distance * np.sin(angle)
        
        # Keep within bounds
        if abs(x) < half_size and abs(y) < half_size:
            pos[node_id] = (x, y)
            G.add_node(node_id)
            G.add_edge(parent, node_id, length=distance)
            
            # Occasionally connect to nearby nodes
            if np.random.random() < 0.3:
                nearby = [n for n in G.nodes() if n != node_id and 
                         np.hypot(pos[n][0] - x, pos[n][1] - y) < 60]
                if nearby:
                    target = np.random.choice(nearby)
                    dist = np.hypot(pos[target][0] - x, pos[target][1] - y)
                    G.add_edge(node_id, target, length=dist)
            
            node_id += 1
    
    nx.set_node_attributes(G, pos, 'pos')
    return G


def create_radial_network(size, n_rings, n_radials):
    """
    Create a radial network (like Paris).
    
    Args:
        size: Area size (meters)
        n_rings: Number of concentric rings
        n_radials: Number of radial streets
    
    Returns:
        NetworkX graph with positions
    """
    G = nx.Graph()
    pos = {}
    node_id = 0
    
    # Create center node
    pos[node_id] = (0, 0)
    center_node = node_id
    node_id += 1
    
    # Create rings and radials
    max_radius = size / 2
    ring_nodes = {0: [center_node]}
    
    for ring in range(1, n_rings + 1):
        radius = (ring / n_rings) * max_radius
        ring_nodes[ring] = []
        
        for radial in range(n_radials):
            angle = 2 * np.pi * radial / n_radials
            x = radius * np.cos(angle)
            y = radius * np.sin(angle)
            
            pos[node_id] = (x, y)
            ring_nodes[ring].append(node_id)
            
            # Connect to inner ring
            if ring > 1:
                inner_node = ring_nodes[ring - 1][radial]
                dist = np.hypot(x - pos[inner_node][0], y - pos[inner_node][1])
                G.add_edge(node_id, inner_node, length=dist)
            elif ring == 1:
                dist = radius
                G.add_edge(node_id, center_node, length=dist)
            
            node_id += 1
        
        # Connect ring nodes
        for i, node in enumerate(ring_nodes[ring]):
            next_node = ring_nodes[ring][(i + 1) % len(ring_nodes[ring])]
            x1, y1 = pos[node]
            x2, y2 = pos[next_node]
            dist = np.hypot(x2 - x1, y2 - y1)
            G.add_edge(node, next_node, length=dist)
    
    nx.set_node_attributes(G, pos, 'pos')
    return G


print("✓ Road network initialization functions defined")

In [ ]:
# Initialize networks for each city with appropriate strategy
print("\n" + "="*60)
print("Initializing road networks...")
print("="*60)

generated_networks = {}

for city_key, city_info in CITIES.items():
    print(f"\n{city_info['name']}:")
    
    target = target_metrics[city_key]
    target_nodes = target['nodes']['total_count']
    target_edges = target['edges']['total_count']
    
    # Choose strategy based on city characteristics
    if city_key == 'london':
        print("  Strategy: Organic growth")
        G = create_organic_network(AREA_SIZE, target_nodes, seed_points=3)
    elif city_key == 'berlin':
        print("  Strategy: Regular grid")
        spacing = np.sqrt((AREA_SIZE ** 2) / target_nodes)
        G = create_grid_network(AREA_SIZE, spacing)
    elif city_key == 'belgrade':
        print("  Strategy: Radial")
        n_rings = int(np.sqrt(target_nodes / 8))
        n_radials = 8
        G = create_radial_network(AREA_SIZE, n_rings, n_radials)
    else:  # torino
        print("  Strategy: Modified grid")
        spacing = np.sqrt((AREA_SIZE ** 2) / target_nodes) * 0.9
        offset = (np.random.uniform(-20, 20), np.random.uniform(-20, 20))
        G = create_grid_network(AREA_SIZE, spacing, offset)
    
    generated_networks[city_key] = {
        'name': city_info['name'],
        'color': city_info['color'],
        'graph': G,
        'target_metrics': target
    }
    
    print(f"  ✓ Initialized: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    print(f"     Target: {target_nodes} nodes, {target_edges} edges")

print("\n" + "="*60)
print("✓ All networks initialized")
print("="*60)

### Visualization: Initial Networks

In [ ]:
# Visualize initial networks
print("\nVisualizing initial networks...")

fig, axes = plt.subplots(1, 4, figsize=(32, 8), facecolor='white')

for idx, (city_key, data) in enumerate(generated_networks.items()):
    ax = axes[idx]
    G = data['graph']
    pos = nx.get_node_attributes(G, 'pos')
    
    # Draw edges
    for u, v in G.edges():
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        ax.plot(x, y, 'k-', linewidth=1.5, alpha=0.6)
    
    # Draw nodes
    x_coords = [pos[n][0] for n in G.nodes()]
    y_coords = [pos[n][1] for n in G.nodes()]
    ax.scatter(x_coords, y_coords, c=data['color'], s=30, alpha=0.8, edgecolors='black', linewidth=0.5, zorder=5)
    
    ax.set_xlim(-RADIUS - 10, RADIUS + 10)
    ax.set_ylim(-RADIUS - 10, RADIUS + 10)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_title(
        f"{data['name']}\n"
        f"{G.number_of_nodes()} nodes, {G.number_of_edges()} edges",
        fontsize=14, fontweight='bold', pad=15
    )
    ax.set_xlabel('X (m)', fontsize=11)
    ax.set_ylabel('Y (m)', fontsize=11)

plt.suptitle('Initial Road Networks (Before Optimization)', fontsize=22, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig(VIZ_PNG_DIR / '01_initial_networks.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.savefig(VIZ_SVG_DIR / '01_initial_networks.svg', bbox_inches='tight', facecolor='white')
plt.show()

print("✓ Saved: 01_initial_networks (PNG + SVG)")

## 4. Network Optimization

Optimize networks to match target metrics from real cities.

In [ ]:
def compute_network_metrics(G):
    """
    Compute metrics for a generated network.
    """
    if G.number_of_nodes() == 0 or G.number_of_edges() == 0:
        return {
            'n_nodes': 0,
            'n_edges': 0,
            'total_length': 0,
            'avg_degree': 0,
            'avg_segment_length': 0
        }
    
    total_length = sum(d['length'] for _, _, d in G.edges(data=True))
    degrees = [d for _, d in G.degree()]
    
    return {
        'n_nodes': G.number_of_nodes(),
        'n_edges': G.number_of_edges(),
        'total_length': total_length,
        'avg_degree': np.mean(degrees),
        'avg_segment_length': total_length / G.number_of_edges() if G.number_of_edges() > 0 else 0
    }


def optimize_network(G, target_metrics, iterations=50):
    """
    Optimize network to match target metrics.
    
    Operations:
    - Add/remove edges to match edge count
    - Adjust node positions to match segment lengths
    - Add shortcuts to match connectivity
    """
    print("  Optimizing network...")
    
    target_edges = target_metrics['edges']['total_count']
    target_length = target_metrics['edges']['total_length_m']
    target_avg_segment = target_metrics['edges']['segment_length_distribution']['mean']
    
    pos = nx.get_node_attributes(G, 'pos')
    
    for iteration in range(iterations):
        current_metrics = compute_network_metrics(G)
        
        # Adjust edge count
        if current_metrics['n_edges'] < target_edges * 0.9:
            # Add edges (shortcuts)
            nodes = list(G.nodes())
            for _ in range(min(5, target_edges - current_metrics['n_edges'])):
                n1, n2 = np.random.choice(nodes, 2, replace=False)
                if not G.has_edge(n1, n2):
                    dist = np.hypot(pos[n1][0] - pos[n2][0], pos[n1][1] - pos[n2][1])
                    if dist < target_avg_segment * 2:  # Only add reasonable shortcuts
                        G.add_edge(n1, n2, length=dist)
        
        elif current_metrics['n_edges'] > target_edges * 1.1:
            # Remove edges
            edges = list(G.edges())
            for _ in range(min(5, current_metrics['n_edges'] - target_edges)):
                edge = edges[np.random.randint(len(edges))]
                # Don't remove if it would disconnect the graph
                G_test = G.copy()
                G_test.remove_edge(*edge)
                if nx.is_connected(G_test):
                    G.remove_edge(*edge)
        
        # Update edge lengths
        for u, v in G.edges():
            dist = np.hypot(pos[u][0] - pos[v][0], pos[u][1] - pos[v][1])
            G[u][v]['length'] = dist
        
        if iteration % 10 == 0:
            current = compute_network_metrics(G)
            print(f"    Iteration {iteration}: {current['n_edges']} edges, "
                  f"{current['total_length']:.0f}m total, "
                  f"{current['avg_segment_length']:.1f}m avg segment")
    
    final_metrics = compute_network_metrics(G)
    print(f"  ✓ Optimization complete:")
    print(f"    Nodes: {final_metrics['n_nodes']} (target: {target_metrics['nodes']['total_count']})")
    print(f"    Edges: {final_metrics['n_edges']} (target: {target_edges})")
    print(f"    Total length: {final_metrics['total_length']:.0f}m (target: {target_length:.0f}m)")
    print(f"    Avg segment: {final_metrics['avg_segment_length']:.1f}m (target: {target_avg_segment:.1f}m)")
    
    return G


print("✓ Optimization functions defined")

In [ ]:
# Optimize all networks
print("\n" + "="*60)
print("Optimizing networks to match target metrics...")
print("="*60)

for city_key, data in generated_networks.items():
    print(f"\n{data['name']}:")
    G = data['graph']
    target = data['target_metrics']
    
    G_optimized = optimize_network(G, target, iterations=OPTIMIZATION_ITERATIONS)
    generated_networks[city_key]['graph'] = G_optimized

print("\n" + "="*60)
print("✓ All networks optimized")
print("="*60)

### Visualization: Optimized Networks

In [ ]:
# Visualize optimized networks
print("\nVisualizing optimized networks...")

fig, axes = plt.subplots(1, 4, figsize=(32, 8), facecolor='white')

for idx, (city_key, data) in enumerate(generated_networks.items()):
    ax = axes[idx]
    G = data['graph']
    pos = nx.get_node_attributes(G, 'pos')
    
    # Draw edges
    for u, v in G.edges():
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        ax.plot(x, y, 'k-', linewidth=2, alpha=0.7)
    
    # Draw nodes
    x_coords = [pos[n][0] for n in G.nodes()]
    y_coords = [pos[n][1] for n in G.nodes()]
    ax.scatter(x_coords, y_coords, c=data['color'], s=40, alpha=0.9, edgecolors='black', linewidth=0.8, zorder=5)
    
    ax.set_xlim(-RADIUS - 10, RADIUS + 10)
    ax.set_ylim(-RADIUS - 10, RADIUS + 10)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    
    metrics = compute_network_metrics(G)
    ax.set_title(
        f"{data['name']}\n"
        f"{metrics['n_nodes']} nodes, {metrics['n_edges']} edges\n"
        f"{metrics['total_length']:.0f}m total length",
        fontsize=14, fontweight='bold', pad=15
    )
    ax.set_xlabel('X (m)', fontsize=11)
    ax.set_ylabel('Y (m)', fontsize=11)

plt.suptitle('Optimized Road Networks', fontsize=22, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig(VIZ_PNG_DIR / '02_optimized_networks.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.savefig(VIZ_SVG_DIR / '02_optimized_networks.svg', bbox_inches='tight', facecolor='white')
plt.show()

print("✓ Saved: 02_optimized_networks (PNG + SVG)")

## 5. Parcel Segmentation

Segment the area into parcels based on the road network.

In [ ]:
def segment_parcels_from_network(G, area_size):
    """
    Segment parcels from road network using polygonization.
    
    Args:
        G: NetworkX graph with 'pos' attributes
        area_size: Size of the area (meters)
    
    Returns:
        GeoDataFrame of parcels
    """
    print("  Segmenting parcels...")
    
    pos = nx.get_node_attributes(G, 'pos')
    half_size = area_size / 2
    
    # Create LineStrings from edges
    lines = []
    for u, v in G.edges():
        line = LineString([pos[u], pos[v]])
        lines.append(line)
    
    # Add boundary box
    boundary = box(-half_size, -half_size, half_size, half_size)
    boundary_lines = [
        LineString([(-half_size, -half_size), (half_size, -half_size)]),
        LineString([(half_size, -half_size), (half_size, half_size)]),
        LineString([(half_size, half_size), (-half_size, half_size)]),
        LineString([(-half_size, half_size), (-half_size, -half_size)])
    ]
    lines.extend(boundary_lines)
    
    # Merge and polygonize
    merged = linemerge(lines)
    if merged.geom_type == 'LineString':
        merged = [merged]
    elif merged.geom_type == 'MultiLineString':
        merged = list(merged.geoms)
    else:
        merged = lines
    
    # Create polygons
    polygons = list(polygonize(merged))
    
    if len(polygons) == 0:
        print("    ⚠ No polygons created, using boundary")
        polygons = [boundary]
    
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': polygons}, crs='EPSG:3857')
    gdf['area'] = gdf.geometry.area
    gdf['perimeter'] = gdf.geometry.length
    gdf['compactness'] = (4 * np.pi * gdf['area']) / (gdf['perimeter'] ** 2)
    gdf['parcel_id'] = [f"parcel_{i:03d}" for i in range(len(gdf))]
    
    print(f"  ✓ Created {len(gdf)} parcels")
    print(f"    Area range: {gdf['area'].min():.0f} - {gdf['area'].max():.0f} m²")
    print(f"    Mean area: {gdf['area'].mean():.0f} m²")
    
    return gdf


print("✓ Parcel segmentation function defined")

In [ ]:
# Segment parcels for all networks
print("\n" + "="*60)
print("Segmenting parcels from road networks...")
print("="*60)

for city_key, data in generated_networks.items():
    print(f"\n{data['name']}:")
    G = data['graph']
    
    parcels = segment_parcels_from_network(G, AREA_SIZE)
    generated_networks[city_key]['parcels'] = parcels
    
    # Save to GeoJSON
    parcels.to_file(GEOJSON_DIR / f"{city_key}_parcels_generated.geojson", driver='GeoJSON')
    print(f"  ✓ Saved to {city_key}_parcels_generated.geojson")

print("\n" + "="*60)
print("✓ All parcels segmented")
print("="*60)

### Visualization: Parcels

In [ ]:
# Visualize parcels
print("\nVisualizing parcels...")

fig, axes = plt.subplots(1, 4, figsize=(32, 8), facecolor='white')

for idx, (city_key, data) in enumerate(generated_networks.items()):
    ax = axes[idx]
    parcels = data['parcels']
    G = data['graph']
    pos = nx.get_node_attributes(G, 'pos')
    
    # Plot parcels
    parcels.plot(ax=ax, color='#B3E5FC', edgecolor='#0277BD', linewidth=1.5, alpha=0.6)
    
    # Plot roads on top
    for u, v in G.edges():
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        ax.plot(x, y, 'k-', linewidth=2, alpha=0.8)
    
    ax.set_xlim(-RADIUS - 10, RADIUS + 10)
    ax.set_ylim(-RADIUS - 10, RADIUS + 10)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_title(
        f"{data['name']}\n"
        f"{len(parcels)} parcels\n"
        f"Mean area: {parcels['area'].mean():.0f}m²",
        fontsize=14, fontweight='bold', pad=15
    )
    ax.set_xlabel('X (m)', fontsize=11)
    ax.set_ylabel('Y (m)', fontsize=11)

plt.suptitle('Parcel Segmentation from Road Networks', fontsize=22, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig(VIZ_PNG_DIR / '03_parcels.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.savefig(VIZ_SVG_DIR / '03_parcels.svg', bbox_inches='tight', facecolor='white')
plt.show()

print("✓ Saved: 03_parcels (PNG + SVG)")

## 6. Building Block Placement Locations

Identify where building blocks should be placed (polygonize only, don't place actual buildings yet).

In [ ]:
def identify_building_block_locations(parcels_gdf, setback=5, min_area=100):
    """
    Identify building block placement locations within parcels.
    
    Args:
        parcels_gdf: GeoDataFrame of parcels
        setback: Setback from parcel boundary (meters)
        min_area: Minimum buildable area (m²)
    
    Returns:
        GeoDataFrame of building block locations
    """
    print("  Identifying building block locations...")
    
    building_blocks = []
    
    for idx, row in parcels_gdf.iterrows():
        parcel = row.geometry
        
        # Apply setback (negative buffer)
        try:
            buildable = parcel.buffer(-setback)
            
            if buildable.is_empty or buildable.area < min_area:
                continue
            
            # Handle MultiPolygon
            if buildable.geom_type == 'MultiPolygon':
                for poly in buildable.geoms:
                    if poly.area >= min_area:
                        building_blocks.append({
                            'geometry': poly,
                            'parcel_id': row['parcel_id'],
                            'buildable_area': poly.area,
                            'coverage_ratio': poly.area / parcel.area
                        })
            elif buildable.geom_type == 'Polygon':
                building_blocks.append({
                    'geometry': buildable,
                    'parcel_id': row['parcel_id'],
                    'buildable_area': buildable.area,
                    'coverage_ratio': buildable.area / parcel.area
                })
        except:
            continue
    
    if len(building_blocks) == 0:
        print("    ⚠ No building blocks identified")
        return gpd.GeoDataFrame()
    
    gdf = gpd.GeoDataFrame(building_blocks, crs='EPSG:3857')
    gdf['block_id'] = [f"block_{i:03d}" for i in range(len(gdf))]
    
    print(f"  ✓ Identified {len(gdf)} building block locations")
    print(f"    Total buildable area: {gdf['buildable_area'].sum():.0f} m²")
    print(f"    Mean coverage ratio: {gdf['coverage_ratio'].mean():.2%}")
    
    return gdf


print("✓ Building block location function defined")

In [ ]:
# Identify building block locations for all networks
print("\n" + "="*60)
print("Identifying building block placement locations...")
print("="*60)

for city_key, data in generated_networks.items():
    print(f"\n{data['name']}:")
    parcels = data['parcels']
    
    building_blocks = identify_building_block_locations(parcels, setback=5, min_area=100)
    generated_networks[city_key]['building_blocks'] = building_blocks
    
    if len(building_blocks) > 0:
        # Save to GeoJSON
        building_blocks.to_file(GEOJSON_DIR / f"{city_key}_building_blocks_generated.geojson", driver='GeoJSON')
        print(f"  ✓ Saved to {city_key}_building_blocks_generated.geojson")

print("\n" + "="*60)
print("✓ All building block locations identified")
print("="*60)

### Visualization: Building Block Locations

In [ ]:
# Visualize building block locations
print("\nVisualizing building block locations...")

fig, axes = plt.subplots(1, 4, figsize=(32, 8), facecolor='white')

for idx, (city_key, data) in enumerate(generated_networks.items()):
    ax = axes[idx]
    parcels = data['parcels']
    building_blocks = data['building_blocks']
    G = data['graph']
    pos = nx.get_node_attributes(G, 'pos')
    
    # Plot parcels (light)
    parcels.plot(ax=ax, color='#E0E0E0', edgecolor='#999999', linewidth=1, alpha=0.3)
    
    # Plot building block locations
    if len(building_blocks) > 0:
        building_blocks.plot(ax=ax, color='#FFD700', edgecolor='#FF8C00', linewidth=1.5, alpha=0.7)
    
    # Plot roads on top
    for u, v in G.edges():
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        ax.plot(x, y, 'k-', linewidth=2, alpha=0.8)
    
    ax.set_xlim(-RADIUS - 10, RADIUS + 10)
    ax.set_ylim(-RADIUS - 10, RADIUS + 10)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    
    total_buildable = building_blocks['buildable_area'].sum() if len(building_blocks) > 0 else 0
    ax.set_title(
        f"{data['name']}\n"
        f"{len(building_blocks)} building blocks\n"
        f"Buildable: {total_buildable:.0f}m²",
        fontsize=14, fontweight='bold', pad=15
    )
    ax.set_xlabel('X (m)', fontsize=11)
    ax.set_ylabel('Y (m)', fontsize=11)

plt.suptitle('Building Block Placement Locations (Gold = Buildable Areas)', fontsize=22, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig(VIZ_PNG_DIR / '04_building_blocks.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.savefig(VIZ_SVG_DIR / '04_building_blocks.svg', bbox_inches='tight', facecolor='white')
plt.show()

print("✓ Saved: 04_building_blocks (PNG + SVG)")

## 7. Space Syntax Analysis of Generated Networks

Analyze generated networks using the same space syntax metrics as STEP 1.

In [ ]:
def analyze_generated_network(G):
    """
    Compute space syntax metrics for generated network.
    """
    print("  Computing space syntax metrics...")
    
    if G.number_of_nodes() == 0 or G.number_of_edges() == 0:
        print("    ⚠ Empty network")
        return None
    
    # Betweenness centrality
    print("    - Betweenness centrality...")
    bc = nx.betweenness_centrality(G, weight='length', normalized=True)
    
    # Closeness centrality
    print("    - Closeness centrality...")
    closeness = nx.closeness_centrality(G, distance='length')
    
    # Degree
    degree = dict(G.degree())
    
    # Edge betweenness
    print("    - Edge betweenness...")
    edge_bc = nx.edge_betweenness_centrality(G, weight='length', normalized=True)
    
    # Store in graph
    nx.set_node_attributes(G, bc, 'betweenness')
    nx.set_node_attributes(G, closeness, 'closeness')
    nx.set_node_attributes(G, degree, 'degree')
    nx.set_edge_attributes(G, edge_bc, 'edge_betweenness')
    
    metrics = {
        'node_integration': np.mean(list(bc.values())),
        'edge_connectivity': np.mean(list(closeness.values())),
        'avg_degree': np.mean(list(degree.values())),
        'max_betweenness': max(bc.values()) if bc else 0,
        'max_edge_betweenness': max(edge_bc.values()) if edge_bc else 0
    }
    
    print("  ✓ Space syntax metrics computed")
    return metrics


print("✓ Space syntax analysis function defined")

In [ ]:
# Analyze all generated networks
print("\n" + "="*60)
print("Analyzing generated networks with space syntax...")
print("="*60)

for city_key, data in generated_networks.items():
    print(f"\n{data['name']}:")
    G = data['graph']
    
    metrics = analyze_generated_network(G)
    generated_networks[city_key]['space_syntax_metrics'] = metrics
    
    if metrics:
        print(f"  Results:")
        print(f"    Node Integration: {metrics['node_integration']:.4f}")
        print(f"    Edge Connectivity: {metrics['edge_connectivity']:.4f}")
        print(f"    Avg Degree: {metrics['avg_degree']:.2f}")

print("\n" + "="*60)
print("✓ All networks analyzed")
print("="*60)

### Visualization: Space Syntax Analysis

In [ ]:
# Visualize node betweenness
print("\nVisualizing node betweenness centrality...")

fig, axes = plt.subplots(1, 4, figsize=(32, 8), facecolor='white')

for idx, (city_key, data) in enumerate(generated_networks.items()):
    ax = axes[idx]
    G = data['graph']
    pos = nx.get_node_attributes(G, 'pos')
    bc = nx.get_node_attributes(G, 'betweenness')
    
    # Draw edges
    for u, v in G.edges():
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        ax.plot(x, y, 'k-', linewidth=1, alpha=0.3)
    
    # Draw nodes sized by betweenness
    if bc:
        x_coords = [pos[n][0] for n in G.nodes()]
        y_coords = [pos[n][1] for n in G.nodes()]
        bc_values = [bc[n] for n in G.nodes()]
        sizes = [(b * 1000) + 10 for b in bc_values]
        
        scatter = ax.scatter(x_coords, y_coords, c=bc_values, s=sizes, 
                            cmap='YlOrRd', alpha=0.8, edgecolors='black', linewidth=0.5, zorder=5)
        plt.colorbar(scatter, ax=ax, label='Betweenness')
    
    ax.set_xlim(-RADIUS - 10, RADIUS + 10)
    ax.set_ylim(-RADIUS - 10, RADIUS + 10)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_title(
        f"{data['name']}\n"
        f"Node Betweenness Centrality",
        fontsize=14, fontweight='bold', pad=15
    )
    ax.set_xlabel('X (m)', fontsize=11)
    ax.set_ylabel('Y (m)', fontsize=11)

plt.suptitle('Space Syntax: Node Betweenness Centrality', fontsize=22, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig(VIZ_PNG_DIR / '05_node_betweenness.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.savefig(VIZ_SVG_DIR / '05_node_betweenness.svg', bbox_inches='tight', facecolor='white')
plt.show()

print("✓ Saved: 05_node_betweenness (PNG + SVG)")

In [ ]:
# Visualize edge betweenness (paths)
print("\nVisualizing edge betweenness (movement corridors)...")

fig, axes = plt.subplots(1, 4, figsize=(32, 8), facecolor='white')

for idx, (city_key, data) in enumerate(generated_networks.items()):
    ax = axes[idx]
    G = data['graph']
    pos = nx.get_node_attributes(G, 'pos')
    edge_bc = nx.get_edge_attributes(G, 'edge_betweenness')
    
    if edge_bc:
        max_bc = max(edge_bc.values())
        
        # Draw edges with width and color based on betweenness
        for (u, v), bc_val in edge_bc.items():
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            width = (bc_val / max_bc) * 5 + 0.5 if max_bc > 0 else 1
            color = plt.cm.YlOrRd(bc_val / max_bc if max_bc > 0 else 0)
            ax.plot(x, y, color=color, linewidth=width, alpha=0.8)
    
    ax.set_xlim(-RADIUS - 10, RADIUS + 10)
    ax.set_ylim(-RADIUS - 10, RADIUS + 10)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_title(
        f"{data['name']}\n"
        f"Edge Betweenness (Movement Corridors)",
        fontsize=14, fontweight='bold', pad=15
    )
    ax.set_xlabel('X (m)', fontsize=11)
    ax.set_ylabel('Y (m)', fontsize=11)

plt.suptitle('Space Syntax: Edge Betweenness (Paths)', fontsize=22, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig(VIZ_PNG_DIR / '06_edge_betweenness.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.savefig(VIZ_SVG_DIR / '06_edge_betweenness.svg', bbox_inches='tight', facecolor='white')
plt.show()

print("✓ Saved: 06_edge_betweenness (PNG + SVG)")

### Visualization: Complete Overlay

In [ ]:
# Create complete overlay visualization
print("\nCreating complete overlay visualization...")

fig, axes = plt.subplots(1, 4, figsize=(32, 8), facecolor='white')

for idx, (city_key, data) in enumerate(generated_networks.items()):
    ax = axes[idx]
    G = data['graph']
    pos = nx.get_node_attributes(G, 'pos')
    parcels = data['parcels']
    building_blocks = data['building_blocks']
    bc = nx.get_node_attributes(G, 'betweenness')
    edge_bc = nx.get_edge_attributes(G, 'edge_betweenness')
    
    # Layer 1: Parcels
    parcels.plot(ax=ax, color='#E8F5E9', edgecolor='#66BB6A', linewidth=0.5, alpha=0.3)
    
    # Layer 2: Building blocks
    if len(building_blocks) > 0:
        building_blocks.plot(ax=ax, color='#FFE082', edgecolor='#FFA000', linewidth=0.8, alpha=0.5)
    
    # Layer 3: Roads (edge betweenness)
    if edge_bc:
        max_bc = max(edge_bc.values()) if edge_bc.values() else 1
        for (u, v), bc_val in edge_bc.items():
            x = [pos[u][0], pos[v][0]]
            y = [pos[u][1], pos[v][1]]
            width = (bc_val / max_bc) * 4 + 1 if max_bc > 0 else 1
            ax.plot(x, y, 'k-', linewidth=width, alpha=0.7)
    
    # Layer 4: Nodes (betweenness)
    if bc:
        x_coords = [pos[n][0] for n in G.nodes()]
        y_coords = [pos[n][1] for n in G.nodes()]
        bc_values = [bc[n] for n in G.nodes()]
        sizes = [(b * 500) + 20 for b in bc_values]
        ax.scatter(x_coords, y_coords, c='#0066CC', s=sizes, 
                  alpha=0.8, edgecolors='white', linewidth=1, zorder=5)
    
    ax.set_xlim(-RADIUS - 10, RADIUS + 10)
    ax.set_ylim(-RADIUS - 10, RADIUS + 10)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_title(
        f"{data['name']}\n"
        f"Complete Overlay: Parcels • Blocks • Roads • Nodes",
        fontsize=14, fontweight='bold', pad=15
    )
    ax.set_xlabel('X (m)', fontsize=11)
    ax.set_ylabel('Y (m)', fontsize=11)

plt.suptitle('OVERLAY: Complete Generated Urban Structure', fontsize=22, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig(VIZ_PNG_DIR / '07_complete_overlay.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.savefig(VIZ_SVG_DIR / '07_complete_overlay.svg', bbox_inches='tight', facecolor='white')
plt.show()

print("✓ Saved: 07_complete_overlay (PNG + SVG)")

## 8. Metrics Comparison: Real vs Generated

In [ ]:
# Create comparison table
print("\n" + "="*80)
print("📊 METRICS COMPARISON: REAL vs GENERATED")
print("="*80)

comparison_data = []

for city_key, data in generated_networks.items():
    target = data['target_metrics']
    generated_metrics = compute_network_metrics(data['graph'])
    
    comparison_data.append({
        'City': data['name'],
        'Real Nodes': target['nodes']['total_count'],
        'Gen Nodes': generated_metrics['n_nodes'],
        'Real Edges': target['edges']['total_count'],
        'Gen Edges': generated_metrics['n_edges'],
        'Real Length (m)': f"{target['edges']['total_length_m']:.0f}",
        'Gen Length (m)': f"{generated_metrics['total_length']:.0f}",
        'Real Parcels': target['parcels']['total_count'],
        'Gen Parcels': len(data['parcels']),
        'Gen Blocks': len(data['building_blocks'])
    })

df = pd.DataFrame(comparison_data)
print("\n" + df.to_string(index=False))

# Save to CSV
df.to_csv(METRICS_DIR / 'comparison_real_vs_generated.csv', index=False)
print(f"\n✓ Saved to comparison_real_vs_generated.csv")
print("="*80)

## 9. Final Summary

In [ ]:
print("\n" + "="*80)
print("✓ STEP 2 COMPLETE: ROAD NETWORK GENERATION")
print("="*80)

print("\n📁 OUTPUTS:")
print(f"  GeoJSON: {len(list(GEOJSON_DIR.glob('*.geojson')))} files")
print(f"  PNG: {len(list(VIZ_PNG_DIR.glob('*.png')))} files")
print(f"  SVG: {len(list(VIZ_SVG_DIR.glob('*.svg')))} files")
print(f"  Metrics: {len(list(METRICS_DIR.glob('*')))} files")

print("\n📊 GENERATION SUMMARY:")
for city_key, data in generated_networks.items():
    G = data['graph']
    metrics = compute_network_metrics(G)
    print(f"\n  {data['name']}:")
    print(f"    Network: {metrics['n_nodes']} nodes, {metrics['n_edges']} edges")
    print(f"    Length: {metrics['total_length']:.0f}m")
    print(f"    Parcels: {len(data['parcels'])}")
    print(f"    Building blocks: {len(data['building_blocks'])}")
    if data['space_syntax_metrics']:
        print(f"    Node integration: {data['space_syntax_metrics']['node_integration']:.4f}")

print("\n✅ DELIVERABLES:")
print("  ✓ Road networks initialized and optimized")
print("  ✓ Parcels segmented from road networks")
print("  ✓ Building block locations identified (polygonized)")
print("  ✓ Space syntax analysis completed")
print("  ✓ All visualizations generated (PNG + SVG)")
print("  ✓ Metrics comparison: Real vs Generated")

print("\n🎯 NEXT STEPS:")
print("  → Place actual building footprints in building blocks")
print("  → Refine building placement rules")
print("  → Generate 3D models")

print("\n" + "="*80)